## Librerías

In [ ]:
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt

import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, Activation


## Directorios y carga de datos

In [ ]:
data_in = "./input/"
data_out = "./output/"

In [ ]:
# Cargamos los datos de las base de datos del Teatro Real

#   - Informacion de los sensores en producción: 
df_hvacs = pd.read_excel(data_in + 'tr_hvacs.xlsx')
#   - Temperaturas tomadas en distintas zonas
df_zones = pd.read_excel(data_in + 'tr_zones.xlsx')
#   - Temeperatura en el auditorio durante los shows
df_shows = pd.read_excel(data_in + 'tr_shows.xlsx')

## Preprocesamiento de datos

In [ ]:
index = pd.date_range('2016-01-01', '2018-03-24 23:45', freq = '15T')

df_shows = df_shows.sort_values(by=['Date'])
df_shows = df_shows.drop_duplicates(['Date'])
df_shows = df_shows.set_index(['Date'])
df_shows = df_shows.reindex(index).fillna(method = 'pad', limit = 1)

df_zones = df_zones.set_index(['Date'])
df_zones = df_zones.reindex(index).interpolate(method = 'linear', limit = 1)

df_hvacs['h_Text'] = df_hvacs['h_Text'].replace(to_replace=0, method='ffill', limit = 2)
df_hvacs = df_hvacs.set_index(['Date'])
df_hvacs = df_hvacs.reindex(index).interpolate(method = 'linear', limit = 1)


In [ ]:
#Concatenamos todos los dataframes en uno solo
df_teatro_real_inicial = pd.concat([df_hvacs, df_shows, df_zones], axis =1)

In [ ]:
#Creamos el DataFrame final que utilizará el modelo
df_teatro_real = pd.DataFrame()
df_teatro_real['Date'] = df_teatro_real_inicial.index
df_teatro_real = df_teatro_real.set_index(['Date'])

#Guardamos las capacidades de los chillers del sistema HVAC

#Ajustamos los valores para que las capacidades de las bombas de frío
#sean de -100 a 0 y las bombas de calor de 0 a 100

df_teatro_real_inicial.loc[df_teatro_real_inicial['h_Wint_ChC'] == 1, 'h_Cap_ChC'] = -df_teatro_real_inicial.loc[df_teatro_real_inicial['h_Wint_ChC'] == 1, 'h_Cap_ChC']
df_teatro_real_inicial.loc[df_teatro_real_inicial['h_Wint_ChF'] == 1, 'h_Cap_ChF'] = -df_teatro_real_inicial.loc[df_teatro_real_inicial['h_Wint_ChF'] == 1, 'h_Cap_ChF']

df_teatro_real['Climatizador 1'] = -df_teatro_real_inicial['h_Cap_Ch1']
df_teatro_real['Climatizador 2'] = -df_teatro_real_inicial['h_Cap_Ch2']
df_teatro_real['Climatizador Carlos'] = -df_teatro_real_inicial['h_Cap_ChC']
df_teatro_real['Climatizador Felipe'] = -df_teatro_real_inicial['h_Cap_ChF']

In [ ]:
df_teatro_real['Modo invierno Carlos'] = df_teatro_real_inicial['h_Wint_ChC'] 
df_teatro_real['Modo invierno Felipe'] = df_teatro_real_inicial['h_Wint_ChF'] 

In [ ]:
#Guardamos en el dataframe del teatro real la energía consumida en los periodos
#Se calcula sumando el coste de todos los chillers y multipliclándolo por
#0.25, ya que son KWh y el periodo consta de 15 minutos 

df_teatro_real['Consumo'] = df_teatro_real_inicial[[
                             'h_Pe_Ch1', 
                             'h_Pe_Ch2', 
                             'h_Pe_ChC', 
                             'h_Pe_ChF']].sum(axis = 1) * 0.25

#Guardamos en el dataframe el coeficiente de rendimiento de los HVAC, 
#que se calcula dividiendo la emisión térmica entre el consumo real

df_teatro_real['COP'] = 0.25*(df_teatro_real_inicial[
                ['h_Wt_Ch1', 
                 'h_Wt_Ch2', 
                 'h_Wt_ChC', 
                 'h_Wt_ChF'
                ]].sum(axis = 1))/(1000*df_teatro_real['Consumo'])

In [ ]:
df_teatro_real['Wt'] = 0.25*(df_teatro_real_inicial[
                ['h_Wt_Ch1', 
                 'h_Wt_Ch2', 
                 'h_Wt_ChC', 
                 'h_Wt_ChF'
                ]].sum(axis = 1))/1000

In [ ]:
#Guardamos en el dataframe del Teatro Real la temperatura interior

df_teatro_real["T_interior_inicial"] = df_teatro_real_inicial[
                                                        [
                                                    's_Tr_AmbC', 
                                                    's_Tr_CrcC', 
                                                    's_Tr_CrcF', 
                                                    's_Tr_FyrF', 
                                                    's_Tr_GdF', 
                                                    's_Tr_GoyaF', 
                                                    's_Tr_Hal1F', 
                                                    's_Tr_PitF', 
                                                    's_Tr_StdsC', 
                                                    's_Tr_StdsF', 
                                                    's_TRet_AmbF', 
                                                    's_TRet_StllC', 
                                                    's_TRet_StllF', 
                                                    'z_Tr_AmbC', 
                                                    'z_Tr_GyrreC', 
                                                    'z_Tr_HalSAPAF', 
                                                    'z_Tr_OrchReheF', 
                                                    'z_Tr_Sng4', 
                                                    'z_TRet_Bllt', 
                                                    'z_TRet_Choir', 
                                                    'z_TRet_CrcC', 
                                                    'z_TRet_CrcF', 
                                                    'z_TRet_Hal6F', 
                                                    'z_TRet_OffiF', 
                                                    'z_TRet_R14', 
                                                    'z_TRet_Store', 
                                                    'z_TRet_Tech'  
                                                        ]
                                                    ].mean(axis = 1, skipna = True)

Valores_temperatura = df_teatro_real['T_interior_inicial'].describe()
iqr = Valores_temperatura['75%'] - Valores_temperatura['25%'] 
iqr_up = Valores_temperatura['75%'] + 1.5 * iqr
iqr_down = Valores_temperatura['25%'] - 1.5 * iqr
df_teatro_real['T_interior_inicial'][(df_teatro_real['T_interior_inicial'] <= iqr_down) | (df_teatro_real['T_interior_inicial'] >= iqr_up)] = df_teatro_real['T_interior_inicial'].rolling(window=10).mean()

#Guardamos la temperatura interior del final del periodo, que corresponde  
#a la temperatura interior inicial del próximo periodo
df_teatro_real['T_interior_final'] = df_teatro_real['T_interior_inicial'].shift(periods = -1)

df_teatro_real['Variacion_temperatura'] = df_teatro_real['T_interior_final'] - df_teatro_real['T_interior_inicial']

In [ ]:
#Guardamos en el dataframe del teatro real la temperatura 
#exterior de cada periodo

df_teatro_real['T_exterior'] = df_teatro_real_inicial['h_Text']

#Guardamos en el dataframe del teatro real el aforo y el numero del evento
df_teatro_real['Evento'] = df_teatro_real_inicial['s_EventOn'].fillna(0)

df_teatro_real['Aforo'] = 0
df_teatro_real.loc[df_teatro_real['Evento'] > 0, 'Aforo'] = 1700


In [ ]:
df_teatro_real.to_csv(data_out + "df_teatro_real.csv")

In [ ]:
df_teatro_real = pd.read_csv(data_out + "df_teatro_real.csv", index_col = 'Date')

In [ ]:
#Se eliminan los registros que no nos sirven
df_teatro_real = df_teatro_real.dropna()
df_teatro_real.loc[df_teatro_real['Consumo'] < 1, 'COP'] = 0
df_teatro_real = df_teatro_real[df_teatro_real['T_interior_inicial']>=15]
df_teatro_real = df_teatro_real[df_teatro_real['T_interior_final']>=15]

In [ ]:
df_teatro_real[['Climatizador 1', 'Climatizador 2', 'Climatizador Carlos', 'Climatizador Felipe', 'Consumo', 'COP', 'Wt', 'T_interior_inicial', 'T_exterior']].describe()

## Fin preprocesamiento general 

### Datos sin modo invierno

In [ ]:
df_teatro_real_sin_invierno = df_teatro_real.copy()

df_teatro_real_sin_invierno.loc[df_teatro_real_sin_invierno['Climatizador Carlos'] > 0, 'Climatizador Carlos'] = -1*df_teatro_real_sin_invierno.loc[df_teatro_real_sin_invierno['Climatizador Carlos'] > 0, 'Climatizador Carlos']
df_teatro_real_sin_invierno.loc[df_teatro_real_sin_invierno['Climatizador Felipe'] > 0, 'Climatizador Felipe'] = -1*df_teatro_real_sin_invierno.loc[df_teatro_real_sin_invierno['Climatizador Felipe'] > 0, 'Climatizador Felipe']


In [ ]:
df_teatro_real_sin_invierno.describe()

### Datos con modo invierno forzado

In [ ]:
df_invierno_f = df_teatro_real.copy()
df_invierno_f.loc[df_invierno_f.index >= '2017-11-01', 'Climatizador Carlos'] = -1*df_invierno_f.loc[df_invierno_f.index >= '2017-11-01', 'Climatizador Carlos']
df_invierno_f.loc[df_invierno_f.index >= '2017-11-01', 'Climatizador Felipe'] = -1*df_invierno_f.loc[df_invierno_f.index >= '2017-11-01', 'Climatizador Felipe']

In [ ]:
df_invierno_f.loc[df_invierno_f.index >= '2017-11-01'].describe()

### Datos con modo invierno al aumentar la temperatura

In [ ]:
df_invierno_aumento = df_teatro_real.copy()

for i in range(7937, 12362):
    if(df_invierno_aumento.iloc[i]['T_interior_final'] > df_invierno_aumento.iloc[i]['T_interior_inicial']):
        df_invierno_aumento.iloc[i, df_invierno_aumento.columns.get_loc('Climatizador Carlos')] =  -df_invierno_aumento.iloc[i]['Climatizador Carlos']
        df_invierno_aumento.iloc[i, df_invierno_aumento.columns.get_loc('Climatizador Felipe')] =  -df_invierno_aumento.iloc[i]['Climatizador Felipe']

In [ ]:
df_teatro_real

## Segmentación de datos para los modelos

In [ ]:
#Dividimos los datos en datos de entrada y salida
entradas_modelo = df_teatro_real[['Climatizador 1',
                                  'Climatizador 2',
                                  'Climatizador Carlos',
                                  'Climatizador Felipe',
                                  'T_interior_inicial',
                                  'T_exterior',
                                  'Aforo'
                                 ]]

salidas_modelo = df_teatro_real[['Consumo',
                                 'Variacion_temperatura',
                                 'Wt'
                                ]]
#Kcal
#Dividimos los datos en datos de entrenamiento y datos de validación
x_train, x_test, y_train, y_test = train_test_split(entradas_modelo, salidas_modelo, test_size=0.2, shuffle = True)


In [ ]:
x_train.describe()

In [ ]:
x_test

In [ ]:
x_train

## Creación del modelo con keras

In [ ]:
#Creamos el perceptrón multicapa con keras y lo entrenamos
model = Sequential([
        Dense(30, input_dim = 7),
        Activation('relu'),
        Dense(20),
        Activation('relu'),
        Dense(10),
        Activation('relu'),
        Dense(3),
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, epochs=35, validation_split=0.3, shuffle=True)

model.save('output/modelo_hvac.h5')

In [ ]:
salidas_test = model.predict(x_test)
score, acc = model.evaluate(x_test, y_test)
print("Error: ", score)
print("Precision: ", acc)

In [ ]:
#GRafica mlp con todos los datos
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(y_test['Variacion_temperatura'], salidas_test[:, 0])
plt.scatter(y_test['Variacion_temperatura'], y_test['Variacion_temperatura'], c='r')
plt.ylabel("Predicción")
plt.xlabel("Valor real")
plt.title("Variación temperatura")

## Creación del modelo con Random Forest

In [ ]:
#Creamos el Random Forest con sklearn
forest = RandomForestRegressor(
    bootstrap = True,
    criterion = 'mse',
    max_depth = None,
    max_features = 'auto',
    max_leaf_nodes = None,
    min_impurity_decrease = 0.0,
    min_impurity_split = None,
    min_samples_leaf = 1,
    min_samples_split = 2,
    min_weight_fraction_leaf = 0.0,
    n_estimators = 20,
    n_jobs = 8,
    oob_score = True,
    random_state = None,
    verbose = 0,
    warm_start = False)

forest.fit(x_train,y_train)

filename = 'forest_model.sav'
pickle.dump(forest, open(filename, 'wb'))

In [ ]:
salidas_test = forest.predict(x_test)
metricas = forest.score(x_test, y_test)
print(metricas)

In [ ]:
plt.figure(figsize=(8, 8), dpi=80)
plt.scatter(y_test['Variacion_temperatura'], salidas_test[:, 1])
plt.scatter(y_test['Variacion_temperatura'], y_test['Variacion_temperatura'], c='r')

In [ ]:
#Obtenemos la importancia de cada entrada para el Random Forest
Comfort_forest_pca = pd.DataFrame(forest.feature_importances_, columns = ['Relevance'], index = list(entradas_modelo.columns)) * 100
Comfort_forest_pca.sort_values(by = 'Relevance', ascending = False, inplace = True)
Comfort_forest_pca

In [ ]:
fig = plt.subplots(2,figsize=(18,7))
plt.rcParams.update({'font.size': 12})
ax = plt.subplot(1, 2, 1)
ax.scatter(y_test["Wt"], prediccion_mlp[:, 2])
ax.scatter(y_test["Wt"], y_test["Wt"], c="r")
ax.set_ylabel("KW")
ax.set_xlabel("KW")
ax.set_title("Perceptrón multicapa")
plt.xticks(rotation=45)
plt.rcParams.update({'font.size': 12})
ay = plt.subplot(1, 2, 2)
ay.scatter(y_test["Wt"], prediccion_forest[:, 2])
ay.scatter(y_test["Wt"], y_test["Wt"], c="r")

ay.set_ylabel("KW")
ay.set_xlabel("KW")
ay.set_title("Random Forest")
plt.xticks(rotation=45)

plt.show()

## Comparamos el error y  R2 de ambos modelos 

In [ ]:
# Cálculo error cuadrático medio
error_consumo_mlp = (abs(prediccion_mlp[:, 0] - y_test['Consumo']).sum())/len(x_test)
error_consumo_forest = (abs(prediccion_forest[:, 0] - y_test['Consumo']).sum())/len(x_test)

print("Error para el consumo: ")
print("\t Perceptron multicapa:", round(error_consumo_mlp, 3), " KW")
print("\t Random Forest: ", round(error_consumo_forest, 3), " KW")

error_temp_mlp = (abs(prediccion_mlp[:, 1] - y_test['Variacion_temperatura']).sum())/len(x_test)
error_temp_forest = (abs(prediccion_forest[:, 1] - y_test['Variacion_temperatura']).sum())/len(x_test)

print("Error para la temperatura: ")
print("\t Perceptron multicapa:", round(error_temp_mlp, 3), " ºC")
print("\t Random Forest: ", round(error_temp_forest, 3), " ºC")

error_wt_mlp = (abs(prediccion_mlp[:, 2] - y_test['Wt']).sum())/len(x_test)
error_wt_forest = (abs(prediccion_forest[:, 2] - y_test['Wt']).sum())/len(x_test)

print("Error para el Wt: ")
print("\t Perceptron multicapa:", round(error_wt_mlp, 3), " KW")
print("\t Random Forest: ", round(error_wt_forest, 3), " KW")

In [ ]:
#Calculo R2

r2_consumo_mlp = metrics.r2_score(y_test['Consumo'], prediccion_mlp[:, 0])
r2_consumo_forest = metrics.r2_score(y_test['Consumo'], prediccion_forest[:, 0])

print("R2 para el consumo: ")
print("\t Perceptron multicapa:", round(r2_consumo_mlp, 3)*100 , "%")
print("\t Random Forest: ", round(r2_consumo_forest, 3)*100, "%")

r2_temperatura_mlp = metrics.r2_score(y_test['Variacion_temperatura'], prediccion_mlp[:, 1])
r2_temperatura_forest = metrics.r2_score(y_test['Variacion_temperatura'], prediccion_forest[:, 1])

print("R2 para la temperatura: ")
print("\t Perceptron multicapa:", round(r2_temperatura_mlp, 3)*100 , "%")
print("\t Random Forest: ", round(r2_temperatura_forest, 3)*100, "%")

r2_wt_mlp = metrics.r2_score(y_test['Wt'], prediccion_mlp[:, 2])
r2_wt_forest = metrics.r2_score(y_test['Wt'], prediccion_forest[:, 2])

print("R2 para la energía suministrada: ")
print("\t Perceptron multicapa:", round(r2_wt_mlp, 3)*100 , "%")
print("\t Random Forest: ", round(r2_wt_forest, 3)*100, "%")

## Preparación datos de simulación

In [ ]:
df_teatro_real["Fecha"] = 0
for i in range(len(df_teatro_real)):
    df_teatro_real['Fecha'].iloc[i] = datetime.datetime.strptime(str(df_teatro_real.iloc[i].name), '%Y-%m-%d %H:%M:%S')
df_teatro_real = df_teatro_real.reset_index(drop = True)

In [ ]:
datos_simulacion = pd.DataFrame()

for i in range (20):
    longitud = 0
    continuar = 0
    while(continuar == 0):
        evento = np.random.randint(73, 310)
        if(len(df_teatro_real[df_teatro_real['Evento'] == evento])>0):
            posicion = df_teatro_real[df_teatro_real['Evento'] == evento].index[0]
            continuar = 1
            if(df_teatro_real["Fecha"].iloc[posicion].hour - df_teatro_real["Fecha"].iloc[posicion-12].hour > 3 or 
              df_teatro_real["Fecha"].iloc[posicion].day != df_teatro_real["Fecha"].iloc[posicion-12].day):
                continuar  = 0
    
    df_aux = df_teatro_real.iloc[posicion-12:posicion]
    datos_simulacion = datos_simulacion.append(
                                pd.DataFrame(
                                {
                                    "Evento" : [evento],
                                    "Hora_inicio": [df_aux["Fecha"].iloc[0]],
                                    "Hora_evento"  : [df_aux["Fecha"].iloc[-1] + datetime.timedelta(minutes=15)],
                                    "T_interior_inicial" : [df_aux["T_interior_inicial"].iloc[0]],
                                    "T_interior_final"   : [df_aux["T_interior_final"].iloc[-1]],
                                    "T_exterior": [df_aux["T_exterior"].reset_index(drop = True)],
                                    "Consumos"  : [df_aux["Consumo"].values],
                                    "Consumo"        : [df_aux["Consumo"].sum()],
                                    "COP"                 : [df_aux["COP"].mean()],
                                    "Confort"             : [df_aux["T_interior_final"].iloc[-1] - 23.5],
                                    "Programa"            : [df_aux[["Fecha", "Climatizador 1", "Climatizador 2", "Climatizador Carlos", "Climatizador Felipe"]].reset_index(drop = True)]
                                }), ignore_index = True) 

In [ ]:
datos_simulacion = datos_simulacion.drop_duplicates(subset = "Hora_inicio")
datos_simulacion.to_csv("datos_simulacion.csv")